# ⭐⭐⭐模型构造：层与块

**基础回顾非常重要**

---

## 一、神经网络到底在干什么？

```
数据进来 → 经过第1层 → 经过第2层 → 经过第3层 → 结果出来
```


## 二、Python基础再rev

### 类class：一张图纸

```python
# "类"就是一个模板/图纸
class Dog:
    def __init__(self, name):      # 造狗时要做的准备
        self.name = name           # 把名字记住（绑在自己身上）
    
    def bark(self):                # 狗会叫
        print("你好，我是" + self.name)

# 用图纸造一只真狗
my_dog = Dog("旺财")
my_dog.bark()                      # 输出：你好，我是旺财
```

| 概念 | 意思 | 比喻 |
|:---|:---|:---|
| `class` | 定义一个模板 | 画图纸 |
| `__init__` | 造东西时要做的准备初始化 | 买材料 |
| `self` | "我自己" | 指代这只狗自己 |
| `self.name` | 我自己的名字 | 这只狗记住自己叫什么 |
| `Dog("旺财")` | 用模板造一个实际的东西 | 按图纸造一只真狗 |

---

## 三、搞懂 self

### 3.1 self = "我自己的"

```python
self.hidden = nn.Linear(20, 256)
# 意思：我自己的hidden层 = 一个吃20吐256的机器
```

### 3.2 为什么需要self？

```python
# ❌ 没有self
def __init__(self):
    hidden = nn.Linear(20, 256)    # 临时变量，函数结束就没了

def forward(self, X):
    X = hidden(X)                  # 报错！hidden已经消失了！
```

```python
# ✅ 有self
def __init__(self):
    self.hidden = nn.Linear(20, 256)    # 绑在自己身上，一直在

def forward(self, X):
    X = self.hidden(X)                  # 能找到！因为绑在身上
```

**生活比喻：**

```
没有self → 在商店买了菜刀，放在门口就走了 → 回家做菜时找不到
有self   → 买了菜刀带回家了 → 随时能用
```

### 3.3 核心规则

**init里用self存东西，forward里用self取东西，名字必须对应！**

```python
# ✅ 名字对应
self.aaa = nn.Linear(20, 256)     # init里叫aaa
X = self.aaa(X)                    # forward里也叫aaa

# ❌ 名字不对应
self.aaa = nn.Linear(20, 256)     # init里叫aaa
X = self.bbb(X)                    # 报错！没有bbb这个东西！
```

---

## 四、搞懂 nn.Linear

### 4.1 它是一台"加工机器"

```python
machine = nn.Linear(20, 256)
```

```
我造了一台机器
    吃 20 个数字（输入维度）
    吐 256 个数字（输出维度）
我给它起名叫 machine
```

**机器被造出来时，PyTorch自动在里面放好了随机的权重，你不需要手动设置。**

### 4.2 数字必须前后对应

**前一层的输出 = 后一层的输入，就像水管接水管：**

```
你的数据
[20个数] ← 输入

    ↓  nn.Linear(20, 256)      20进，256出

[256个数]

    ↓  nn.Linear(256, 10)      256进，10出
                                这个256必须=上面的256！

[10个数] ← 最终输出
```

```
如果不匹配：
machine1 吐出 256 个数
machine2 只能吃 100 个数
→ 报错！塞不进去！就像粗水管接细水管，接不上！
```

---

## 五、搞懂 F

### 5.1 F是什么？

```python
from torch.nn import functional as F
```

**F就是一个装满现成函数的工具包，起名叫F只是为了少打字。**

```
F.relu()       → ReLU激活函数（负数变0）
F.sigmoid()    → Sigmoid函数
F.softmax()    → Softmax函数
```

### 5.2 F.relu vs nn.ReLU

```
F.relu(X)     → 直接说"帮我切菜"        → 一个动作
nn.ReLU()     → 先买把菜刀放厨房再切     → 一个东西
```

**结果完全一样！ReLU没有要学习的参数，所以用F.relu更省事。**

**简单规则：**

| 情况 | 用什么 | 例子 |
|:---|:---|:---|
| 有参数要学习的层 | nn.XXX | nn.Linear |
| 没参数的纯计算 | F.xxx | F.relu |

---

## 六、哪些能改，哪些不能改

```python
class MLP(nn.Module):
#     ^^^ 你起的名字 ✅能改     
#          nn.Module  ❌不能改（PyTorch规定）

    def __init__(self):          # __init__ ❌不能改（Python规定）
        super().__init__()       # ❌不能改（固定写法）
        self.hidden = nn.Linear(20, 256)
#            ^^^^^^ 你起的名字 ✅能改
#                    nn.Linear ❌不能改（PyTorch提供的层）
#                              ^^ ^^^ 数字 ✅根据需要改
    
    def forward(self, X):        # forward ❌不能改（PyTorch规定）
#                     ^ 你起的名字 ✅能改
        X = self.hidden(X)
#            ^^^^^^ 必须和init里的名字对应！
        X = F.relu(X)            # F.relu ❌不能改（PyTorch提供的函数）
```

---

## 七、两种构造方式

### 7.1 方式一：Sequential（简单版）

**像坐地铁：只能按站走，不能拐弯。**

```python
net = nn.Sequential(
    nn.Linear(20, 256),    # 第1站
    nn.ReLU(),             # 第2站
    nn.Linear(256, 10)     # 第3站
)

X = torch.rand(2, 20)     # 造2个样本，每个20个特征
output = net(X)            # 丢进去，自动按顺序走
```

### 7.2 方式二：自定义类（灵活版）

**像自己开车：想怎么走就怎么走。**

```python
class MLP(nn.Module):
    def __init__(self):     # 我有哪些零件
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):   # 数据怎么走
        X = self.hidden(X)
        X = F.relu(X)
        X = self.out(X)
        return X

net = MLP()      # 造网络【实例化】
output = net(X)    # 丢数据（自动调用forward）
```

### 7.3 什么时候用哪个？

| 场景 | 用什么 |
|:---|:---|
| 简单的一层接一层 | Sequential，省事 |
| 需要if判断、循环、分支等 | 自定义类 |

---

## 八、nn.Module 是一切的基础

```
nn.Linear      → 是 nn.Module 的子类
nn.ReLU        → 是 nn.Module 的子类  
nn.Sequential  → 是 nn.Module 的子类
你自定义的MLP   → 也是 nn.Module 的子类
```

**都是同一个接口，所以可以像乐高一样随意拼接嵌套：**

```python
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()#必须写！！！
        self.net = nn.Sequential(  # 里面放Sequential
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)  # 再加一个单独的层
    
    def forward(self, X):
        return self.linear(self.net(X))   # 先过net再过linear
        # 一定⭐⭐注意执行顺序前向传播由内向外的顺序！
```

**大盒子里放小盒子，小盒子里还能放更小的盒子。**

---

## 再谈困惑


## 问题：init和forward有区别吗？init里面写了层是怎么传比如hidden 和out为什么forward里面还要写？

### 用做菜来讲

**做菜分两步：**

```
第1步（买菜）：去超市买好所有食材
第2步（炒菜）：决定先放什么后放什么，怎么炒
```

**这两步能合成一步吗？不能！**

你不能一边买菜一边炒菜，必须先买好，再按顺序炒。

### init = 买菜初始化准备

```python
def __init__(self):
    self.hidden = nn.Linear(20, 256)   # 买了一口锅，叫hidden
    self.out = nn.Linear(256, 10)      # 买了一把铲子，叫out
```

**现在有这两个网络了！！！只是买好工具放家里，还没开始做菜！**

### forward = 炒菜【正式开始前向传播】

```python
def forward(self, X):          # X就是你要炒的菜（原材料）
    X = self.hidden(X)         # 第1步：把菜放进锅里（用hidden处理）
    X = F.relu(X)              # 第2步：翻炒一下（用relu处理）
    X = self.out(X)            # 第3步：用铲子盛出来（用out处理）
    return X                   # 第4步：上菜！
```

**forward决定了做菜的顺序和步骤！**

### 为什么不能只写init？

**因为光买了工具，PyTorch不知道你要怎么用啊！**

```python
# init里你买了 hidden 和 out
# 但是PyTorch不知道：
#   先用hidden还是先用out？
#   中间要不要加relu？
#   要不要跳过某一步？

# 所以你必须在forward里告诉它顺序！
```

### 如果你买了3个工具，可以有不同的做菜方式

```python
# init：买了3个工具
self.layer1 = nn.Linear(20, 64)
self.layer2 = nn.Linear(64, 32)
self.layer3 = nn.Linear(32, 10)

# forward方式A：按顺序全用
X = self.layer1(X)
X = self.layer2(X)
X = self.layer3(X)

# forward方式B：跳过第2个
X = self.layer1(X)
# 不用layer2
X = self.layer3(X)

# forward方式C：layer1用两次
X = self.layer1(X)
X = self.layer1(X)    # 再过一遍！
X = self.layer3(X)
```

**同样的工具，不同的用法！所以必须分开写！**

---

## 问题3：forward里每一行到底在干什么？

### 再谈数据如何流动！注意形状！

**起点：X是你的原始数据，形状是 2×20（2个样本，每个20个数字）**

```
X = [0.3, 0.7, 0.1, 0.5, ... 共20个数字]    ← 样本1
    [0.8, 0.2, 0.4, 0.9, ... 共20个数字]    ← 样本2
```

---

**第1行：`X = self.hidden(X)`**

```
X（20个数字）丢进 hidden 这台机器

hidden机器是 nn.Linear(20, 256)
它会把 20个数字 变成 256个数字

出来之后：
X = [0.12, -0.34, 0.56, ... 共256个数字]    ← 样本1
    [0.78, -0.91, 0.23, ... 共256个数字]    ← 样本2

注意：X被覆盖了！
旧X（20个数）已经没了
新X（256个数）取代了它
```

---

**第2行：`X = F.relu(X)`**

```
relu做的事：负数变成0，正数不变

之前：X = [0.12, -0.34, 0.56, -0.91, ...]
之后：X = [0.12,  0,    0.56,  0,    ...]
                  ^^^^         ^^^^
                  负数变0了     负数变0了

数字的个数没变，还是256个
只是把负数清零了
```

---

**第3行：`X = self.out(X)`**

```
X（256个数字）丢进 out 这台机器

out机器是 nn.Linear(256, 10)
它会把 256个数字 变成 10个数字

出来之后：
X = [0.45, 0.12, -0.33, ... 共10个数字]    ← 样本1
    [0.67, -0.22, 0.89, ... 共10个数字]    ← 样本2
```

---

**第4行：`return X`**

```
把最终的X（10个数字）返回给调用者
完成！
```

### 完整流程一张图

```
原始数据 X
[2×20]  每个样本20个数字
   |
   ↓  self.hidden(X)     hidden机器：20→256
   |
[2×256] 每个样本256个数字
   |
   ↓  F.relu(X)          负数变0
   |
[2×256] 每个样本还是256个（只是负数没了）
   |
   ↓  self.out(X)        out机器：256→10
   |
[2×10]  每个样本10个数字
   |
   ↓  return X           吐出结果
```

---

**理解逻辑，逻辑永远是这样：**

```
第1步：买工具（init）
第2步：用工具做菜（forward）
第3步：上菜（return）
```

**每次写代码就想这三步：**

```
我需要哪些层？         → 在init里写
数据先过哪层再过哪层？  → 在forward里写
最后把结果return出去
```

### 固定的壳子（每次都一样）

```python
class 随便起个名字(nn.Module):
    
    def __init__(self):
        super().__init__()
        # 在这里列出你需要的层
    
    def forward(self, X):
        # 在这里写数据怎么一步步走
        return X
```

### 然后你只需要填空

```python
class MyNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self._____ = nn.Linear(__, __)    # 填：层的名字和大小
        self._____ = nn.Linear(__, __)    # 填：层的名字和大小
    
    def forward(self, X):
        X = self._____(X)                  # 填：第1步用哪个层
        X = F.relu(X)                      # 填：要不要加激活函数
        X = self._____(X)                  # 填：第2步用哪个层
        return X
```

---

## 最终总结

### init和forward的关系

```
init   =  买好所有工具放家里
forward =  决定怎么用这些工具、用的顺序

不重复！一个是"有什么"，一个是"怎么用"
```
### 需要记住的

```
✅ init里：用self.名字 = nn.Linear(进, 出) 绑好层
✅ forward里：用self.名字(X) 按顺序使用层
✅ 前一层的"出" = 后一层的"进"
✅ return把结果返回
✅ 反向传播不用管
```

---

## 十、完整代码逐行翻译

```python
import torch                            # 导入PyTorch工具箱
from torch import nn                    # 拿出神经网络工具
from torch.nn import functional as F    # 拿出函数工具包，叫F

class MLP(nn.Module):                   # 画图纸，叫MLP

    def __init__(self):                 # 准备工作
        super().__init__()              # PyTorch自己的准备（固定写法）
        self.hidden = nn.Linear(20, 256)  # 绑一台机器：吃20吐256
        self.out = nn.Linear(256, 10)     # 绑一台机器：吃256吐10

    def forward(self, X):               # 数据怎么走
        X = self.hidden(X)              # X过hidden层：20→256
        X = F.relu(X)                   # 负数变0
        X = self.out(X)                 # X过out层：256→10
        return X                        # 吐出结果

net = MLP()                             # 按图纸造网络
X = torch.rand(2, 20)                   # 造数据：2个样本×20特征
output = net(X)                         # 丢进网络，输出2×10
```
